# Part 1: Imports, data uploads and preparation.

As is customary, let us first call the Python libraries needed here, and upload the needed data and code.

In [ ]:
from model import setup, balance_calcs, performance, visuals
import copy
import pandas as pd
import numpy as np
import rhodium
from matplotlib import pyplot as plt
import datetime

## Loading model and water balance for historical data

Insert the name of **your coursework reservoir** below.

In [ ]:
# Preparing the model
reservoir_name = 'coursework'
downstream_demand_names = ['Environmental']
direct_demand_names = ['Clean water', 'Irrigation']  # With intake and demand specifications

In [ ]:
# Loading the model!
coursework = setup.define_reservoir(reservoir_name, downstream_demand_names, direct_demand_names)

# Read flow and demand data
flows_init = setup.extract_flows(reservoir=coursework)

In [ ]:
# Computing the water balance for our standard operating policy (SOP)
sop_balance = flows_init.copy()
sop_balance = balance_calcs.sop_full(coursework, sop_balance)
display(sop_balance.head())

## Motivation for changing the operating policy

In [ ]:
# Let us create a time series of hydropower releases
sop_hp_release = np.minimum(sop_balance['Release (m3/s)'].values, np.ones(len(sop_balance))*coursework.hydropower_plant.max_release)

# We plot hydropower releases vs. total releases. What do we see?
fig = visuals.compare_flow_timeseries(reference=pd.Series(sop_balance['Release (m3/s)']),
                                      alternative=pd.Series(sop_hp_release, index=sop_balance.index),
                                      labels=['Total release', 'Hydropower release'], 
                                      first_date=datetime.date(1990, 1, 1), 
                                      last_date=datetime.date(1991, 1, 1))

>* **Question 1. What is the extra release? What does that mean for hydropower production under SOP?**

Note that you can change the dates in the plot above to visualise other years. You'll see that even very dry years (such as 1964) still have significant extra release.


# Part 2: Storage targets

We see that if we let the reservoir fill up entirely, sometimes we have no choice but to let water be spilled, i.e., not having any meaningful use. To mitigate this, we define a policy such that **when storage is above the target we can release more than specified by SOP, and produce hydropower.**
Otherwise we stick to the standard operating policy (SOP), and avoid extra releases to protect supply. Targets can change every month because the need for conserving water changes during the year, it is stronger in summer than in winter months.

To implement this target, we use method `monthly_storage_targets` from `balance_calcs.py`. Have a look at the method to understand what it does: you can look at the text at the top of the function, and / or copy and paste the method into Gemini and ask for explanation on any point that's unclear.

>* **Question 2. The standard operating policy (SOP) is equivalent to setting storage targets at which level?**

In [ ]:
# Testing
my_balance = flows_init.copy()
balance_calcs.monthly_storage_targets(reservoir=coursework, 
                                      water_flows=my_balance,
                                      monthly_target=np.ones(12)*coursework.full_lake_volume)
print(my_balance.equals(sop_balance))

## 2.1 - Optimization

Now we can define our optimization of monthly storage targets. **Note that since this tutorial does not model firm power, I multiplied irrigation by 3.** This multiplier would need to be brought back to 1 if firm power is integrated.

In [ ]:
def optimization_problem(storage_target,
                         irrigation_multiplier=3.0):
    """
    Simulator for the performance of the reservoir if monthly storage targets are implemented,
    and irrigation area increased.
    :param storage_target: numpy vector, 12 long, one value for each month (in m3)
    :param irrigation_multiplier: float, the quantity by which to multiply irrigation demand
    :return: reliability of irrigation demand (0-1 scale), average daily hydropower production (MWh)
    """

    # Get copies of the data so that there is an untouched original copy
    balance_table = flows_init.copy()
    reservoir = copy.deepcopy(coursework)
    balance_table['Irrigation demand (m3/s)'] = balance_table['Irrigation demand (m3/s)'] * irrigation_multiplier

    # Computing the water balance for our policy
    balance_calcs.monthly_storage_targets(reservoir=reservoir, water_flows=balance_table, monthly_target=storage_target)

    # We compute performance indicators
    # Global reliability
    rel = performance.reliability(balance_table['Withdrawals Irrigation (m3/s)'],
                                  balance_table['Irrigation demand (m3/s)'], above_desirable=True)
    # Hydropower production: average daily production (MWh)
    hp_average = reservoir.daily_production(balance_table).mean()

    return rel, hp_average

In [ ]:
# Get SOP results with 3 times the irrigation (baseline)
baseline_results = optimization_problem(np.ones(12)*coursework.full_lake_volume)
print('Irrigation reliability is ' + "{:.3f}".format(baseline_results[0]) + '.')
print('Average daily hydropower production over 70 years is ' + "{:.1f}".format(baseline_results[1]) + ' MWh.')

Next we define the model as per the nomenclature of the Rhodium library. Note the constraint on reliability.

In [ ]:
# Let us now define the model for the Rhodium library
storage_targets = rhodium.Model(optimization_problem)

# Model parameters
storage_targets.parameters = [rhodium.Parameter('storage_target'),
                              rhodium.Parameter('irrigation_multiplier')]

# Model levers. Notice how these can be different from the parameters
storage_targets.levers = [rhodium.RealLever('storage_target', coursework.dead_storage, coursework.full_lake_volume,
                                            length=12)]

# Model responses, i.e., the metrics we seek to maximize / minimize as objectives.
storage_targets.responses = [rhodium.Response('rel', rhodium.Response.MAXIMIZE),
                             rhodium.Response('hp_average', rhodium.Response.MAXIMIZE)]

# Constraint: we want reliability over a threshold
storage_targets.constraints = [rhodium.Constraint('rel > 0.90')]

Next, perform the optimization (if needed) or upload a typical results CSV, provided with the tutorial material (note it is not necessarily tailored 100% to your data, but it enables you first to run the tutorial without the optimization).

In [ ]:
# Switch optimization to 1 to run it.
optimization = 0
results_path = 'results/optimized_storage_targets.csv'

if optimization == 1:
    # Optimize
    output = rhodium.optimize(storage_targets, "NSGAII", 5000)
    print("Found", len(output), "optimal policies!")
    #Save results
    df_results = output.as_dataframe()
    df_results.to_csv(results_path)
else:
    # Load results
    df_results = pd.read_csv(results_path, index_col=0)#, dtype={'storage_target': list, 'rel': np.float64, 'hp_average': np.float64})    

# 2.2 - Policy design

In [ ]:
fig = plt.figure(figsize=(14,8))
ax = fig.add_subplot(1, 1, 1)
ax.scatter(baseline_results[0], baseline_results[1], color='r', marker='s', label='SOP')
ax.scatter(df_results['rel'], df_results['hp_average'], label='Storage targets')
ax.legend(prop={'size': 14})
ax.set_xlabel('Irrigation reliability', size=16)
ax.set_ylabel('Average daily hydropower (MWh)', size=16)
ax.tick_params(axis='both', which='major', labelsize=14)

>* **Question 3. What solution do you decide to select and why?**

Let us visualise what some of the solutions mean in terms of storage targets.

In [ ]:
sol_nb = df_results['rel'].idxmax()
# If you are loading the results from a CSV, it read a string, and we need to convert into a vector. If you used optimization, you don't need to convert.
if isinstance(df_results.loc[sol_nb, 'storage_target'], str):
    monthly_targets = np.array(df_results.loc[sol_nb, 'storage_target'][1:-1].split(','), dtype=float)
else:
    monthly_targets = df_results.loc[sol_nb, 'storage_target']

fig = plt.figure(figsize=(14,8))
ax = fig.add_subplot(1, 1, 1)
ax.bar(np.arange(1,13), monthly_targets, label='Target storage')
ax.set_xticks(ticks=np.arange(1, 13, 1), labels=['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D'])
ax.plot(np.arange(0, 14, 1), coursework.full_lake_volume * np.ones(14), c='black', linewidth=2, label='Max storage')
ax.plot(np.arange(0, 14, 1), coursework.dead_storage * np.ones(14), c='black', linestyle='--', linewidth=2, label='Dead storage')
ax.set_ylabel('Storage (m3)', size=16)
ax.set_xlabel('Month', size=16)
ax.set_xlim(0.4, 12.6)
ax.tick_params(axis='both', which='major', labelsize=14)

>* **Question 4. Outputs from MOEAs are random up to a point. How can we translate that into a release policy design?** 

In [ ]:
# Let's implement this, e.g., for irrigation we may have
design_targets = np.array(monthly_targets)  # Initialise

# Beginning of year
design_targets[0:4] = np.mean(monthly_targets[0:4])

# End of year
design_targets[8:12] = np.mean(monthly_targets[8:12])

In [ ]:
# Let's visualise the result!
fig = plt.figure(figsize=(14,8))
ax = fig.add_subplot(1, 1, 1)
ax.bar(np.arange(1,13), design_targets, label='Target storage')
ax.set_xticks(ticks=np.arange(1, 13, 1), labels=['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D'])
ax.plot(np.arange(0, 14, 1), coursework.full_lake_volume * np.ones(14), c='black', linewidth=2, label='Max storage')
ax.plot(np.arange(0, 14, 1), coursework.dead_storage * np.ones(14), c='black', linestyle='--', linewidth=2, label='Dead storage')
ax.set_ylabel('Storage (m3)', size=16)
ax.set_xlabel('Month', size=16)
ax.set_xlim(0.4, 12.6)
ax.tick_params(axis='both', which='major', labelsize=14)

>* **Question 5. How do you explain the seasonal variations in storage targets?** 

In [ ]:
# How does that affect performance?
raw_results = optimization_problem(monthly_targets)
design_results = optimization_problem(design_targets)
print('Irrigation reliability is ' + "{:.3f}".format(design_results[0]) + ' for the design vs. ' 
      + "{:.3f}".format(raw_results[0]) + ' for the raw optimization result.')
print('Average daily hydropower production over 70 years is ' + "{:.1f}".format(design_results[1]) + ' MWh for the design policy vs. ' 
      + "{:.1f}".format(raw_results[1]) + ' MWh for the raw optimization result.')

>* **Question 6. What do you make of the discrepancy between the two?**